# Sample Text2Cypher Agent Walkthrough

This notebook will walk users through setting up a Text2Cypher Agent that leverages the [Reactome Pathway Database](https://reactome.org/dev/graph-database)

This agent utilizes Amazon Neptune to store this graph data.

# Pre-requisites

1. Go through the notebook environment setup in the agents_catalog/0-Notebook-environment/ folder

2. Deploy text2cypher_infra.yaml to your AWS account to instantiate an Amazon Neptune instance, ensure that the SageMaker notebook instance you are running this notebook in is in the same VPC as the deployed Neptune database

#### Ensure the latest version of boto3 is shown below

In [1]:
!pip freeze | grep boto3

boto3==1.37.17


In [3]:
import boto3

In [7]:
# boto3 session
sts_client = boto3.client('sts')
session = boto3.session.Session()

# Account info
account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

# FM used for all agents, choose cross-region if needed
agent_foundation_model = ["anthropic.claude-3-5-sonnet-20241022-v2:0"]
# agent_foundation_model = ["us.anthropic.claude-3-5-sonnet-20241022-v2:0"]

# Supervisor agent FM, choose cross-region if needed
supervisor_agent_foundation_model = ["anthropic.claude-3-5-sonnet-20241022-v2:0"]
# supervisor_agent_foundation_model = ["us.anthropic.claude-3-5-sonnet-20241022-v2:0"]

In [12]:
print(region)

us-west-2


#### Retrieve imports environment variable and bring libraries into notebook
#### Load in environment variables to notebook

In [8]:
%run $IMPORTS_PATH

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[03/21/25 14:25:47] INFO     Found credentials from IAM Role:                                   ]8;id=820044;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=883121;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
Successfully imported necessary libraries into notebook


## Extract information from CloudFormation Stack Outputs
### Replace the "stack_name" variable with the name you put when deploying 'text2cypher_infra.yaml'

In [10]:
stack_name = "neptune"

In [11]:
import boto3

def get_stack_outputs(stack_name, region='us-west-2'):
    cloudformation = boto3.client('cloudformation', region_name=region)
    
    try:
        # Get stack outputs
        response = cloudformation.describe_stacks(StackName=stack_name)
        outputs = response['Stacks'][0]['Outputs']
        
        # Convert outputs to dictionary
        output_dict = {output['OutputKey']: output['OutputValue'] for output in outputs}
        
        return output_dict
    
    except Exception as e:
        print(f"Error getting stack outputs: {str(e)}")
        return None

# Usage
outputs = get_stack_outputs(stack_name)

# Extract all necessary outputs from CloudFormation stack
if outputs:
    SERVER = outputs['DBClusterEndpoint']
    PORT = 8182
    ROLE_ARN = outputs['NeptuneLoadFromS3IAMRoleArn']
    BUCKET_NAME = outputs['AmazonS3BucketName']
    NEPTUNE_VPC = outputs['VPC']
    NEPTUNE_SG = outputs['NeptuneSG']
    NEPTUNE_READER_ENDPOINT = outputs['DBClusterReadEndpoint']
    
    print(PORT)
    print(SERVER)
    print(ROLE_ARN)
    print(BUCKET_NAME)
    print(NEPTUNE_VPC)
    print(NEPTUNE_SG)
    print(NEPTUNE_READER_ENDPOINT)


8182
reactome-db-048051882663-us-west-2.cluster-c9kuqm8skznm.us-west-2.neptune.amazonaws.com
arn:aws:iam::048051882663:role/neptune-NeptuneLoadFromS3Role-BNr2BLxRhaTa
reactome-data-048051882663-us-west-2
vpc-0b8fef50a2c256b19
sg-0c78114358c16a773
reactome-db-048051882663-us-west-2.cluster-ro-c9kuqm8skznm.us-west-2.neptune.amazonaws.com


In [ ]:
import boto3

# Initialize S3 client
s3 = boto3.client('s3')

# Define bucket and file key directly
reactome_data_bucket_name = 'aws-hcls-ml'
file_name = 'reactome/reactome_data.zip'

try:
    # Download file
    s3.download_file(reactome_data_bucket_name, file_name, 'reactome_data.zip')
    print('Reactome data downloaded successfully')
except Exception as e:
    print(f'Error downloading file: {e}')

## Extract Reactome data from provided .zip file

In [13]:
import zipfile

# Unzip data 
with zipfile.ZipFile('reactome_data.zip', 'r') as zip_ref:
    zip_ref.extractall()
    
print("Unzipped Reactome Graph data")

# Upload to S3 bucket for later ingestion
s3_client = boto3.client('s3')

# List all files in the data folder
files = os.listdir('reactome_data')

# Upload each file to S3 bucket
for file in files:
    local_file_path = os.path.join('reactome_data', file)
    
    try:
        print(f"Uploading {file} to S3...")
        s3_client.upload_file(local_file_path, BUCKET_NAME, file)
        print(f"Successfully uploaded {file}")
    except Exception as e:
        print(f"Error uploading {file}: {str(e)}")

print(f"Upload to bucket {BUCKET_NAME} complete!")

Unzipped Reactome Graph data
Uploading vertices.csv to S3...
Successfully uploaded vertices.csv
Uploading edges.csv to S3...
Successfully uploaded edges.csv
Upload to bucket reactome-data-048051882663-us-west-2 complete!


## Load Reactome data into Amazon Neptune
Note: This can take a very long time! Do not proceed in notebook until this data load is complete. Check the neptune_load.log for detailed information on how the data load is progressing!

In [14]:
import os
import json
import datetime
import time
import requests

ENDPOINT = f"https://{SERVER}:{PORT}/loader"

headers = {'Content-type': 'application/json'}
data = {
    "source": f"s3://{BUCKET_NAME}/",
    "format": "csv",
    "iamRoleArn": ROLE_ARN,
    "region": region,
    "failOnError": "TRUE",
    "parallelism": "HIGH",
    "updateSingleCardinalityProperties": "FALSE",
    "queueRequest": "TRUE",
    "mode": "NEW"
}

# Function to write logs to a file
def write_to_log(message, filename="neptune_load.log"):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    try:
        if isinstance(message, dict):
            message = json.dumps(message, indent=2)
        
        # Create directory if it doesn't exist
        log_dir = os.path.dirname(filename)
        if log_dir and not os.path.exists(log_dir):
            os.makedirs(log_dir)
            
        with open(filename, "a", encoding='utf-8') as log_file:
            log_file.write(f"[{timestamp}] {message}\n")
            log_file.flush()
            os.fsync(log_file.fileno())  # Ensure write to disk
    except Exception as e:
        print(f"Error writing to log: {str(e)}")

# Start of the script
write_to_log("Starting Neptune load job")

resp = requests.post(ENDPOINT, data=json.dumps(data), headers=headers)

print(resp.json())
write_to_log(f"Initial response: {resp.json()}")

# Extract the load ID from the response
load_id = resp.json().get('payload', {}).get('loadId')

if load_id:
    write_to_log(f"Load ID: {load_id}")
    # Check the status of the load job
    status_endpoint = f"https://{SERVER}:{PORT}/loader/{load_id}"
    
    # Poll the status until the job is complete
    while True:
        status_resp = requests.get(status_endpoint, headers=headers)
        status = status_resp.json().get('payload', {}).get('overallStatus', {}).get('status')
        
        print(f"Current status: {status}")
        write_to_log(f"Status check - Current status: {status}")

        # Get detailed information about the load job
        details_endpoint = f"https://{SERVER}:{PORT}/loader/{load_id}?details=true"
        details_resp = requests.get(details_endpoint, headers=headers)
        details = details_resp.json()
        
        write_to_log(f"Detailed status: {details}")

        if status == 'LOAD_CANCELLED_DUE_TO_ERRORS':
            errors_endpoint = f"https://{SERVER}:{PORT}/loader/{load_id}?details=true&errors=true"
            errors_resp = requests.get(errors_endpoint, headers=headers)
            print(json.dumps(errors_resp.json(), indent=2))
            write_to_log(f"Load job errors: {json.dumps(errors_resp.json())}")
        
        if status in ['LOAD_COMPLETED', 'LOAD_FAILED']:
            print(f"Final load job status: {status}")
            write_to_log(f"Final load job status: {status}")
            break
   
        time.sleep(30)  # Wait for 30 seconds before checking again
   
else:
    print("Failed to get load ID. Check the initial response.")
    write_to_log("Failed to get load ID. Check the initial response.")

# End of the script
print("Neptune load job completed")
write_to_log("Neptune load job completed")


{'status': '200 OK', 'payload': {'loadId': '04c6949f-b926-42ef-9e58-aa49b76e1da1'}}
Current status: LOAD_IN_PROGRESS
Current status: LOAD_IN_PROGRESS
Current status: LOAD_IN_PROGRESS
Current status: LOAD_IN_PROGRESS
Current status: LOAD_IN_PROGRESS
Current status: LOAD_IN_PROGRESS
Current status: LOAD_IN_PROGRESS
Current status: LOAD_IN_PROGRESS
Current status: LOAD_IN_PROGRESS
Current status: LOAD_IN_PROGRESS
Current status: LOAD_IN_PROGRESS
Current status: LOAD_IN_PROGRESS
Current status: LOAD_IN_PROGRESS
Current status: LOAD_IN_PROGRESS
Current status: LOAD_IN_PROGRESS
Current status: LOAD_IN_PROGRESS
Current status: LOAD_IN_PROGRESS
Current status: LOAD_COMPLETED
Final load job status: LOAD_COMPLETED
Neptune load job completed


# Create Text2Cypher Agent
In this section we create the Text2Cypher agent

#### Fill out your agent configuration below 

In [15]:
agent_name = 'biological-pathways-agent'
agent_description = 'This agent interacts with Reactome data by converting text to openCypher queries for Amazon Neptune'
agent_instruction = """
You are a medical research assistant AI specialized in generating and running openCypher queries for a Reactome graph database
containing human biological pathways and reactions. Your primary task is to interpret user queries, generate appropriate 
openCypher queries, and run the openCypher queries against the Neptune database and provide relevant biological insights 
based on the data. Use only the appropriate tools as required by the specific question. Follow these instructions carefully: 
1. Before generating any openCypher query, use the /getschema tool to familiarize yourself with the property and types of 
nodes and the edges. This will ensure your queries are correctly formatted and target the appropriate nodes and edges in the 
graph. 2. When generating an openCypher query: a. Write the query as a single line, removing all newline ("") characters. 
b. Property names should remain consistent, do not modify the property names in the generated openCypher query. 
3. proceed to execute the query using the /queryneptune tool after generating the openCypher query 
4. Always provide a response to the user. If you can't get results tell the user as such: 
a. Start with a brief summary of your understanding of the user's query. 
b. Explain the steps you're taking to address the query. 
c. Ask for clarifications from the user if required. 
d. If the query fails to execute or doesn't retrieve results, ask the user for the stable identifier of the protein/pathway 
related to the question. 
Here are some examples of question and the corresponding cypher query: 
Question: a. Which molecules participate in Interleukin-4 and 13 signaling (R-HSA-6785807)? 
OpenCypher query: MATCH (p:Pathway{stId:"R-HSA-6785807"})- [:hasEvent*]->(rle:ReactionLikeEvent), 
(rle)-[: input|output|catalystActivity|entityFunctionalStatus|physicalEntity|regulatedBy| 
regulator|hasComponent/hasMember|hasCandidate|repeatedUnit*] ->(pe:PhysicalEntity), (pe)-[:referenceEntity]-›
(re:ReferenceEntity)-[:referenceDatabase]->(rd:ReferenceDatabase) RETURN DISTINCT re. identifier AS Identifier, 
rd.displayName AS Database Question: b. In which pathways does CCR5 (UniProt:P51681) participate? 
OpenCypher query: MATCH (p:Pathway)-[:hasEvent*] ->(rle:ReactionLikeEvent), 
(rle)-[:input|output|catalystActivity|entityFunctionalStatus|physicalEntity|regulatedBy| 
regulator|hasComponent|hasMember|hasCandidate|repeatedUnit*]->(pe:PhysicalEntity), 
(pe)-[: referenceEntity]->(re:ReferenceEntity(identifier:"P51681"}), (re) -[:referenceDatabase]->
(rd:ReferenceDatabase{displayName:"UniProt"}) RETURN DISTINCT p.stId AS Identifier, p.displayName AS Pathway
"""

#### Instantiate your agent with the desired configuration

In [16]:
agents = AgentsForAmazonBedrock()

text2cypher_agent = agents.create_agent(
    agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    code_interpretation=False,
    verbose=False
)

text2cypher_agent

('XNKNELHZ6U',
 'TSTALIASID',
 'arn:aws:bedrock:us-west-2:048051882663:agent-alias/XNKNELHZ6U/TSTALIASID')

#### Extract useful agent information

In [17]:
text2cypher_agent_id = text2cypher_agent[0]
text2cypher_agent_arn = f"arn:aws:bedrock:{region}:{account_id}:agent/{text2cypher_agent_id}"

text2cypher_agent_id, text2cypher_agent_arn

('XNKNELHZ6U', 'arn:aws:bedrock:us-west-2:048051882663:agent/XNKNELHZ6U')

#### Define the API Schema needed for an ActionGroup
##### A simple sample schema is provided below

In [18]:
api_schema_string = '''
  {
  "openapi": "3.0.1",
  "info": {
    "title": "Database schema look up and query APIs",
    "version": "1.0.0",
    "description": "APIs for looking up graph database schemas and making queries to graph database."
  },
  "paths": {
    "/getschema": {
      "get": {
        "summary": "Get a list of property and type of nodes and edges in the Neptune database",
        "description": "Get the list of property and type of nodes and edges in the Neptune database. Return all the property information in graph database.",
        "operationId": "getschema",
        "responses": {
          "200": {
            "description": "Gets the list of property and type of nodes and edges in the Neptune database",
            "content": {
              "application/json": {
                "schema": {
                  "type": "string",
                  "description": "A string containing graph schema information"
                }
              }
            }
          }
        }
      }
    },
    "/queryneptune": {
      "get": {
        "summary": "API to send query to the Neptune database",
        "description": "Send a query to the Neptune graph database to retrieve information pertaining to the users question. The API takes in only one openCypher query at a time, sends the openCypher statement and returns the query results from the graph database. This API should be called for each openCypher query to a Neptune database.",
        "operationId": "queryneptune",
        "parameters": [
          {
            "name": "query",
            "in": "query",
            "required": true,
            "schema": {
              "type": "string"
            },
            "description": "openCypher statement to query Neptune database."
          }
        ],
        "responses": {
          "200": {
            "description": "Query sent successfully",
            "content": {
              "application/json": {
                "schema": {
                  "type": "object",
                  "properties": {
                    "responseBody": {
                      "type": "string",
                      "description": "The query response from the Neptune database."
                    }
                  }
                }
              }
            }
          },
          "400": {
            "description": "Bad request. One or more required fields are missing or invalid."
          }
        }
      }
    }
  }
}
'''

api_schema = {"payload": api_schema_string}

#### Create Lambda function for use by agent ActionGroup

#### Attach Lambda function and create ActionGroup

In [19]:
# Define Lambda func. details
text2cypher_lambda_function_name = "text2cypher"
text2cypher_lambda_function_arn = f"arn:aws:lambda:{region}:{account_id}:function:{text2cypher_lambda_function_name}"
text2cypher_lambda_function_arn

'arn:aws:lambda:us-west-2:048051882663:function:text2cypher'

In [20]:
agents.add_action_group_with_lambda(
    agent_name=agent_name,
    lambda_function_name=text2cypher_lambda_function_name,
    source_code_file="text2cypher_lambda.py",
    agent_action_group_name="Text2CypherActionGroup",
    agent_action_group_description="This tool generates Cypher queries and return a response from an Amazon Neptune database with Reactome data to a user's natural language query",
    api_schema=api_schema,
    verbose=True
)

Creating action group: Text2CypherActionGroup...
Lambda ARN: arn:aws:lambda:us-west-2:048051882663:function:text2cypher
Agent functions: None


In [21]:
lambda_permissions = '''{{
    "Version": "2012-10-17",
    "Statement": [
        {{
            "Effect": "Allow",
            "Action": "logs:CreateLogGroup",
            "Resource": "arn:aws:logs:us-west-2:{account_id}:*"
        }},
        {{
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": [
                "arn:aws:logs:us-west-2:{account_id}:log-group:/aws/lambda/*:*"
            ]
        }},
        {{
            "Sid": "DBAccess",
            "Effect": "Allow",
            "Action": "neptune-db:*",
            "Resource": [
                "arn:aws:neptune-db:us-west-2:{account_id}:*/*"
            ]
        }},
        {{
            "Effect": "Allow",
            "Action": [
                "ec2:CreateNetworkInterface",
                "ec2:DescribeNetworkInterfaces",
                "ec2:DeleteNetworkInterface",
                "ec2:AssignPrivateIpAddresses",
                "ec2:UnassignPrivateIpAddresses",
                "bedrock:*"
            ],
            "Resource": "*"
        }},
        {{
            "Action": [
                "bedrock:InvokeModel"
            ],
            "Resource": "arn:aws:bedrock:us-west-2::foundation-model/{agent_foundation_model[0]}",
            "Effect": "Allow",
            "Sid": "BedrockAccess"
        }},
        {{
            "Sid": "AllowS3Put",
            "Effect": "Allow",
            "Action": [
                "s3:PutObject",
                "s3:GetObject"
            ],
            "Resource": [
                "arn:aws:s3:::*/*"
            ]
        }}
    ]
}}'''

# Replace the variables in the policy
filled_policy = lambda_permissions.format(
    account_id=account_id,
    agent_foundation_model=agent_foundation_model
)

# Parse the string to ensure it's valid JSON and format it nicely
formatted_policy = json.dumps(json.loads(filled_policy), indent=4)

# Print the formatted policy
print(formatted_policy)

{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "logs:CreateLogGroup",
            "Resource": "arn:aws:logs:us-west-2:048051882663:*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": [
                "arn:aws:logs:us-west-2:048051882663:log-group:/aws/lambda/*:*"
            ]
        },
        {
            "Sid": "DBAccess",
            "Effect": "Allow",
            "Action": "neptune-db:*",
            "Resource": [
                "arn:aws:neptune-db:us-west-2:048051882663:*/*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "ec2:CreateNetworkInterface",
                "ec2:DescribeNetworkInterfaces",
                "ec2:DeleteNetworkInterface",
                "ec2:AssignPrivateIpAddresses",
                "ec2:Un

#### Attach policy above as an inline policy to the Lambda function's execution role

In [22]:
# Initialize AWS clients
iam = boto3.client('iam')
lambda_client = boto3.client('lambda')

try:
    # Get the Lambda function configuration to find its role
    lambda_config = lambda_client.get_function(FunctionName='text2cypher')
    role_arn = lambda_config['Configuration']['Role']
    role_name = role_arn.split('/')[-1]

    # Your policy document (assuming filled_policy contains the policy JSON)
    formatted_policy = json.dumps(json.loads(filled_policy), indent=4)

    # Create the policy
    policy_name = 'Text2CypherPolicy'
    policy_response = iam.create_policy(
        PolicyName=policy_name,
        PolicyDocument=formatted_policy,
        Description='Policy for text2cypher Lambda function'
    )

    # Attach the policy to the role
    iam.attach_role_policy(
        RoleName=role_name,
        PolicyArn=policy_response['Policy']['Arn']
    )

    print(f"Successfully attached policy to role {role_name}")

except Exception as e:
    print(f"Error attaching policy: {str(e)}")

Successfully attached policy to role biological-pathways-agent-lambda-role-us-west-2-048051882663


#### Add the Lambda layer to the function

In [23]:
# Publish the Lambda layer
with open('lambda_layer.zip', 'rb') as zip_file:
    zip_binary = zip_file.read()

layer_response = lambda_client.publish_layer_version(
    LayerName='Text2CypherLayer',
    Description='Custom layer for text2cypher function',
    Content={
        'ZipFile': zip_binary
    }
)

layer_version_arn = layer_response['LayerVersionArn']

# Update the Lambda function to use the layer
lambda_client.update_function_configuration(
    FunctionName='text2cypher',
    Layers=[layer_version_arn]
)

print(f"Layer added successfully to function {text2cypher_lambda_function_name}")

Layer added successfully to function text2cypher


#### Increase Lambda function timeout to 15 minutes

In [24]:
# Update the Lambda function to have 15 minute timeout
lambda_client.update_function_configuration(
    FunctionName='text2cypher',
    Timeout=900  # 15 minutes = 900 seconds
)

print("Successfully increased lambda function timeout")

Successfully increased lambda function timeout


#### Add Lambda function environment variables and create new query results S3 bucket

In [25]:
random_suffix = uuid.uuid4().hex[:6]

# Create Neptune query results bucket
bucket_name = f"{'text2cypher-agent'}-{account_id}-{random_suffix}"

s3_client = boto3.client('s3')
s3 = boto3.resource('s3')

if region == 'us-east-1':
    # For us-east-1, don't specify LocationConstraint
    s3_client.create_bucket(Bucket=bucket_name)
else:
    s3_client.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={'LocationConstraint': region}
    )

print(f"Created bucket: {bucket_name}")

lambda_client.update_function_configuration(
    FunctionName=text2cypher_lambda_function_name,
    Environment={
        'Variables': {
            'NEPTUNE_HOST': NEPTUNE_READER_ENDPOINT,
            'NEPTUNE_PORT': str(PORT),
            'BUCKET_NAME': bucket_name
        }
    }
)
print(f"Environment variables added successfully to {text2cypher_lambda_function_name}")

Created bucket: text2cypher-agent-048051882663-626dd0
Environment variables added successfully to text2cypher


#### Lambda function VPC configuration to contact Amazon Neptune

In [26]:
ec2_client = boto3.client('ec2')

# First get the subnets
response = ec2_client.describe_subnets(
    Filters=[{'Name': 'vpc-id', 'Values': [NEPTUNE_VPC]}]
)

subnets_by_az = {}
for subnet in response['Subnets']:
    az = subnet['AvailabilityZone']
    if az not in subnets_by_az:
        subnets_by_az[az] = subnet['SubnetId']

subnet_ids = list(subnets_by_az.values())

# Create new security group
sg_response = ec2_client.create_security_group(
    GroupName='bedrock-vpc-endpoint-sg',
    Description='Security group for Bedrock VPC endpoint',
    VpcId=NEPTUNE_VPC
)
security_group_id = sg_response['GroupId']

# Add inbound rule to allow all traffic
ec2_client.authorize_security_group_ingress(
    GroupId=security_group_id,
    IpPermissions=[
        {
            'IpProtocol': '-1',  # -1 means all protocols
            'FromPort': -1,      # -1 means all ports
            'ToPort': -1,
            'UserIdGroupPairs': [{'GroupId': NEPTUNE_SG}]  # Reference the source security group
        }
    ]
)

# Create VPC endpoint with new security group
response = ec2_client.create_vpc_endpoint(
    VpcEndpointType='Interface',
    VpcId=NEPTUNE_VPC,
    ServiceName=f"com.amazonaws.{region}.bedrock-runtime",
    SubnetIds=subnet_ids,
    SecurityGroupIds=[security_group_id],
    PrivateDnsEnabled=True
)

print(f"Created VPC endpoint: {response['VpcEndpoint']['VpcEndpointId']}")
print(f"Created security group: {security_group_id}")

Created VPC endpoint: vpce-0b8b1ae7e38e4d701
Created security group: sg-0fb3e328078c1d0df


In [27]:
ec2 = boto3.client('ec2')

# Update Lambda VPC configuration
lambda_client.update_function_configuration(
    FunctionName=text2cypher_lambda_function_name,
    VpcConfig={
        'SubnetIds': subnet_ids,
        'SecurityGroupIds': [NEPTUNE_SG]
    }
)
print(f"VPC configuration for Lambda function {text2cypher_lambda_function_name} updated successfully")

VPC configuration for Lambda function text2cypher updated successfully


#### Add resource based policy to Lambda function to allow agent to invoke

In [28]:
lambda_client = boto3.client('lambda', region)

# Define the resource policy statement
policy_statement = {
    "Sid": "AllowBedrockAgentAccess",
    "Effect": "Allow",
    "Principal": {
        "Service": "bedrock.amazonaws.com"
    },
    "Action": "lambda:InvokeFunction",
    "Resource": text2cypher_lambda_function_arn,
    "Condition": {
        "ArnEquals": {
            "aws:SourceArn": text2cypher_agent_arn
        }
    }
}

try:
    # Get the current policy
    response = lambda_client.get_policy(FunctionName=text2cypher_lambda_function_arn)
    current_policy = json.loads(response['Policy'])
    
    # Add the new statement to the existing policy
    current_policy['Statement'].append(policy_statement)
    
except lambda_client.exceptions.ResourceNotFoundException:
    # If there's no existing policy, create a new one
    current_policy = {
        "Version": "2012-10-17",
        "Statement": [policy_statement]
    }

# Convert the policy to JSON string
updated_policy = json.dumps(current_policy)

# Add or update the resource policy
response = lambda_client.add_permission(
    FunctionName=text2cypher_lambda_function_arn,
    StatementId="AllowText2CypherAgentAccess",
    Action="lambda:InvokeFunction",
    Principal="bedrock.amazonaws.com",
    SourceArn=text2cypher_agent_arn
)

print("Resource policy added successfully.")
print("Response:", response)

Resource policy added successfully.
Response: {'ResponseMetadata': {'RequestId': '8a000071-bb96-49f8-afa7-229d81c99bf9', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Fri, 21 Mar 2025 14:56:03 GMT', 'content-type': 'application/json', 'content-length': '353', 'connection': 'keep-alive', 'x-amzn-requestid': '8a000071-bb96-49f8-afa7-229d81c99bf9'}, 'RetryAttempts': 0}, 'Statement': '{"Sid":"AllowText2CypherAgentAccess","Effect":"Allow","Principal":{"Service":"bedrock.amazonaws.com"},"Action":"lambda:InvokeFunction","Resource":"arn:aws:lambda:us-west-2:048051882663:function:text2cypher","Condition":{"ArnLike":{"AWS:SourceArn":"arn:aws:bedrock:us-west-2:048051882663:agent/XNKNELHZ6U"}}}'}


#### Test the Text2Cypher Agent in the AWS Bedrock console
Step 1: Navigate to the Bedrock console

Step 2: Click 'Agents' and select the 'sample-text2cypher-agent'

Step 3: Ask any of the questions below and get an answer!

#### Sample Questions

1. "Which molecules participate in Interleukin-4 and Interleukin-13 signaling?"
2. "What proteins are associated with Signaling by EGFR in Cancer (R-HSA-1643713)?"
3. "What other molecules participate in Signaling by EGFR in Cancer (R-HSA-1643713)?"
4. "What diseases are impacted with Signaling by EGFR in Cancer (R-HSA-1643713)?"
5. "What drugs impact the Signaling by EGFR in Cancer (R-HSA-1643713)?"
6. "What are the immediate downstream reactions that are triggered by EGFRvIII in the 'Signaling by EGFRvIII in Cancer (R-HSA-5637812)' pathway in Homo sapiens?"
7. "What proteins are associated with Signaling by EGFR in Cancer (R-HSA-1643713)?"
8. "Which chemical compounds are involved in Mitochondrial translation termination (R-HSA-5419276)?"
9. "What are 5 different types of proteins involved with the SARS-CoV-2 Infection (R-HSA-9694516)?"
10. "Which events are in the Cap-dependent Translation Initiation pathway?"

#### Now that agent has been tested via direct invoke, prepare it by creating an alias

In [29]:
text2cypher_agent_alias_id, text2cypher_agent_alias_arn = agents.create_agent_alias(
    text2cypher_agent[0], 'v1'
)
%store text2cypher_agent_alias_arn

text2cypher_agent_alias_id, text2cypher_agent_alias_arn

Stored 'text2cypher_agent_alias_arn' (str)


('AGC98M3UQL',
 'arn:aws:bedrock:us-west-2:048051882663:agent-alias/XNKNELHZ6U/AGC98M3UQL')